# Linear Regression with CalCOFI: Over 60 years of oceanographic data

<figure style="display: block; margin: 0 auto; text-align:center;">
<img src="../resources/david-vives-aWDI-dp0gJQ-unsplash.jpg" 
        alt="Photo from David Vives in Unsplash" />
        <figcaption>Photo from <a href="https://unsplash.com/@davidvives?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">David Vives</a> in <a href="https://unsplash.com/es/s/fotos/ocean-california?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Unsplash</a></figcaption>
</figure>

CalCOFI's Bottle Database is one of the most extensive datasets about oceanographic data collected from chemical analyses of seawater samples (1949 - present). The dataset is made-up of two tables in two files respectively. The first, `cast` table contains metadata for each CTD cast ever completed on a CalCOFI cruise. And the second, the `bottle` table contains oceanographic measurements for each bottle/sampling depth ever completed on a CalCOFI cruise [[1]](#database). 

## Problem Statement

Suppose your are an engineer at CalCOFI, recently there have been some problems with the CTD cast at CalCOFI stations. As a result, the oxygen samples (drawn from the [niskin bottles](https://calcofi.org/sampling-info/methods/bottle-sampling-methods/#dissolved-oxygen)) were contaminated by atmosferic oxygen; Such a thing makes the information unreliable. Henceforth, our Data Management Lead requested a **ML model to estimate the oxygen's data** in the samples based on all the previously collected data in order to fill up the missing values in the new data to prepare them for integration in the bottle dataset. We need to estimate the following parameters of our data:
- **O2ml_L**: Milliliters oxygen per liter of seawater.
- **O2Sat**: Oxygen percent saturation.
- **Oxy_µmol/Kg**: Oxygen micromoles per kilogram seawater.

## Downloading the dataset

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from kaggle_utils.utils import KaggleUtils

In [ ]:
dataset_name = 'vikrishnan/boston-house-prices'
with KaggleUtils() as api:
    # api.kaggle_download_dataset(dataset_name)
    pass

## Data Preparation and Cleaning

For our analysis, we are going to use the dataset contained in the file `housing.csv`, which was downloaded from Kaggle. We need to prepare our data by filling up any missing values. First we need to create a pandas dataframe using the downloaded file.

In [ ]:
import pandas as pd

In [ ]:
bottle_df = pd.read_csv("bottle.csv", low_memory=False)

In [8]:
bottle_df.head()

,Cst_Cnt,Btl_Cnt,Sta_ID,Depth_ID,Depthm,T_degC,Salnty,O2ml_L,STheta,O2Sat,...,R_PHAEO,R_PRES,R_SAMP,DIC1,DIC2,TA1,TA2,pH2,pH1,DIC Quality Comment
0,1,1,054.0 056.0,19-4903CR-HY-060-0930-05400560-0000A-3,0,10.50,33.440,NaN,25.649,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,054.0 056.0,19-4903CR-HY-060-0930-05400560-0008A-3,8,10.46,33.440,NaN,25.656,NaN,...,NaN,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,3,054.0 056.0,19-4903CR-HY-060-0930-05400560-0010A-7,10,10.46,33.437,NaN,25.654,NaN,...,NaN,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,4,054.0 056.0,19-4903CR-HY-060-0930-05400560-0019A-3,19,10.45,33.420,NaN,25.643,NaN,...,NaN,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,5,054.0 056.0,19-4903CR-HY-060-0930-05400560-0020A-7,20,10.45,33.421,NaN,25.643,NaN,...,NaN,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
bottle_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864863 entries, 0 to 864862
Data columns (total 74 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Cst_Cnt              864863 non-null  int64  
 1   Btl_Cnt              864863 non-null  int64  
 2   Sta_ID               864863 non-null  object 
 3   Depth_ID             864863 non-null  object 
 4   Depthm               864863 non-null  int64  
 5   T_degC               853900 non-null  float64
 6   Salnty               817509 non-null  float64
 7   O2ml_L               696201 non-null  float64
 8   STheta               812174 non-null  float64
 9   O2Sat                661274 non-null  float64
 10  Oxy_µmol/Kg          661268 non-null  float64
 11  BtlNum               118667 non-null  float64
 12  RecInd               864863 non-null  int64  
 13  T_prec               853900 non-null  float64
 14  T_qual               23127 non-null   float64
 15  S_prec           

In [11]:
bottle_df.isna().sum()

Cst_Cnt                     0
Btl_Cnt                     0
Sta_ID                      0
Depth_ID                    0
Depthm                      0
                        ...  
TA1                    862779
TA2                    864629
pH2                    864853
pH1                    864779
DIC Quality Comment    864808
Length: 74, dtype: int64

In [ ]:
corr_df = bottle_df.corr(numeric_only=True)
O2ml_corr = corr_df[["O2ml_L"]]

In [ ]:
O2ml_corr.loc[(O2ml_corr["O2ml_L"] > 0.5) | (O2ml_corr["O2ml_L"] < -0.5)]

## References
1. <a name="database" id="database">[CalCOFI Bottle Database](https://calcofi.org/data/oceanographic-data/bottle-database/)</a>
2. <a name="sampling" id="sampling">[Dissolved Oxygen Sampling Method](https://calcofi.org/sampling-info/methods/bottle-sampling-methods/#dissolved-oxygen)</a>
3. 
4. 
5. 